In [1]:
using Pkg
using LinearAlgebra
using Plots
using Distributions
using Random
using StateSpaceDynamics
using StatsBase

const SSD = StateSpaceDynamics

StateSpaceDynamics

In [2]:
"""
Create an underlying GaussianHMM to generate data
"""

# Create Guassian Emission Models
output_dim = 2
μ = [0.0, 0.0]
Σ = 0.1 * Matrix{Float64}(I, output_dim, output_dim)
emission_1 = GaussianEmission(Gaussian(output_dim=output_dim, μ=μ, Σ=Σ))

μ = [2.0, 1.0]
Σ = 0.1 * Matrix{Float64}(I, output_dim, output_dim)
emission_2 = GaussianEmission(Gaussian(μ=μ, Σ=Σ, output_dim=output_dim))

# Create GaussianHMM
true_model = SSD.GaussianHMM(K=2, output_dim=2)
true_model.B[1] = emission_1
true_model.B[2] = emission_2
true_model.A = [0.9 0.1; 0.8 0.2]

# Sample from the model
n=1000
true_labels, data = SSD.sample(true_model, n=n)

([1, 1, 1, 1, 1, 1, 1, 2, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [-0.24987588432479124 -0.04689425260460763; 0.007459913013807148 -0.16555198059425713; … ; 0.12041547747978548 -0.041644650267985944; -0.5015985870405232 0.12172859381133226])

In [3]:
"""
Create a new GaussianHMM and try to fit the synthetic data
"""
n = 10  # Number of samples per trial
num_trials = 3  # Number of trials
trial_inputs = Vector{Matrix{Float64}}(undef, num_trials)  # Vector to store data matrices
trial_labels = Vector{Vector{Int}}(undef, num_trials)  # Vector to store label vectors
trial_outputs = Vector{Matrix{Float64}}(undef, num_trials)

for i in 1:num_trials
    true_labels, data = SSD.sample(true_model, n=n)  # Generate data and labels
    trial_labels[i] = true_labels  # Store labels for the ith trial
    trial_inputs[i] = data  # Store data matrix for the ith trial

    true_labels, data = SSD.sample(true_model, n=n)  # Generate data and labels
    trial_outputs[i] = data  # Store data matrix for the ith trial
end

In [6]:
"""
Generate a multiple trial GaussianHMM Dataset from the fit model
"""
model2 =SSD.GaussianHMM(K=2, output_dim=2)
ll = SSD.fit!(model2, trial_inputs, trials=true, max_iters=100)

74-element Vector{Float64}:
 -Inf
 -80.02593616709049
 -53.71576399874769
 -53.50268023616104
 -53.157603159417455
 -52.69831364030862
 -52.284161756845606
 -51.99661524077764
 -51.80185127049626
 -51.66162220200639
   ⋮
 -48.580042895490145
 -48.379716384478066
 -48.330421238078216
 -48.32261519313336
 -48.32168500502476
 -48.32158177837702
 -48.321569460001385
 -48.32156773359358
 -48.32156743805467

In [7]:
model3 = SSD.SwitchingGaussianRegression(K=2, input_dim=2, output_dim=2)
ll2 = SSD.fit!(model3, trial_inputs, trial_outputs, trials=true, max_iters=100)

10-element Vector{Float64}:
 -Inf
 -98.00143926441129
 -74.19960357423763
 -74.54636202011973
 -69.15361231393082
 -63.24368692678478
 -46.223622363438004
  73.31887967204617
 102.8519763643304
 102.8519763643304

In [20]:
"""
Test Creation for Trialized SwitchingGaussianRegression
"""

# Create a true underlying model
model = SSD.SwitchingGaussianRegression(K=2, input_dim=1, output_dim=1)
model.B[1].β = [5; 2;;]
model.B[2].β = [-3; -2;;]

# Define initial state probabilities (π) and transition matrix (A)
initial_probs = [0.6, 0.4]  # Probability to start in state 1 or state 2
transition_matrix = [0.9 0.1; 0.4 0.6]  # State transition probabilities

n = 100 # Number of samples per trial
num_trials = 50  # Number of trials

# Vectors to store generated data
trial_inputs = Vector{Matrix{Float64}}(undef, num_trials)
trial_labels = Vector{Vector{Int}}(undef, num_trials)
trial_outputs = Vector{Matrix{Float64}}(undef, num_trials)

# Function to sample from initial state and transition matrix
function sample_states(num_samples, initial_probs, transition_matrix)
    states = Vector{Int}(undef, num_samples)
    states[1] = rand(Categorical(initial_probs))  # Initial state
    for i in 2:num_samples
        states[i] = rand(Categorical(transition_matrix[states[i - 1], :]))  # State transitions
    end
    return states
end

# Generate trials
for trial in 1:num_trials
    # Random input data
    x_data = randn(n, 1)  # Random input data for this trial
    trial_inputs[trial] = x_data

    # Generate state sequence
    state_sequence = sample_states(n, initial_probs, transition_matrix)
    trial_labels[trial] = state_sequence

    # Generate output data based on state and linear relationships
    y_data = zeros(n, 1)
    for i in 1:n
        if state_sequence[i] == 1
            y_data[i] = model.B[1].β[2] * x_data[i] + model.B[1].β[1]  # y = 2x + 5
        else
            y_data[i] = model.B[2].β[2] * x_data[i] + model.B[2].β[1]  # y = -2x - 3
        end
    end
    trial_outputs[trial] = y_data
end


println("Generated Inputs: ", trial_inputs)
println("Generated State Labels: ", trial_labels)
println("Generated Outputs: ", trial_outputs)


est_model = SSD.SwitchingGaussianRegression(K=2, input_dim=1, output_dim=1)
ll = SSD.fit!(est_model, trial_inputs, trial_outputs, trials=true)



Generated Inputs: [[-0.7493457453322258; -0.34356677778934414; -0.06521984815822147; 1.3241482866145438; 0.5427909334893629; 0.7482999469028873; -0.9381082816050662; -1.4664043996465161; -0.38088049167465865; 0.08268715302131366; -0.5277545027619043; 0.740044935567554; 0.08544993273920347; 0.18769993265711155; 0.17273725319444538; -0.8123134514932655; 0.07542334745964241; 0.1849132370028138; -0.9110284040986533; -0.31216756523033445; -0.056359315157382045; 0.33018576556136053; -1.9307519114545022; 0.7238754761824026; -0.11453448666337276; 0.8326667368419157; 0.24310247787784225; -0.04690142393567391; 1.7852042030550592; 1.977689313501973; -1.8191746440743242; 1.1939598855338853; 1.112673367904612; 0.1035821166425151; -2.1146899622288546; 0.034958237375951266; 0.6962977810931015; 1.0981053758935846; -1.4729153812313482; -0.8654857356785293; -0.6233788676220642; -0.0873400557334861; 0.3845401403528546; 0.46147556592966116; 1.362822360630966; -0.9268506115650783; -2.6811375531943007; 1.50

Excessive output truncated after 524305 bytes.

-1.4806877166915138; 1.234857621164523; 0.3761401864353455; -1.722776922806069; -1.5920064192590448; 1.6146383046771875; -0.45472488045592563; -1.9933188832853752; -1.5902266064172215; -0.06674039196931297; 0.9059552126928473; 0.6480759968127835; 1.7710550305849158; 0.9114020811013748; -0.41291595566186406; 1.547312897488983; -0.5571255375405815; 0.46575193353547983; 1.3684640437558733; -0.37864694695653384; 0.7051327361539188; -0.40131004077237303; -0.41886552333190924; -0.678762348605965; 0.1876305983817442; 0.6099896629751155; -0.35793701899616337; 0.11331792501115452; -0.7621864243202741; -0.035280042456078395; -0.8470775922855812; -0.5833838887798501; -0.008839266136548892; -0.7941806420587797; -1.3167562597905178; -1.3255069458456235; -0.5656972464608082; 0.03705822973224907; 0.8445057659114266; 1.747199584404487; 0.40303315688599767; 0.5525729202918322; -0.3247506779480127; 1.9897913212178058; -0.010464305123883761; -0.00751240145515601; -1.9237813539017918; -1.11696380505374; -

10-element Vector{Float64}:
    -Inf
 -73441.26725561554
 -18554.01525141398
 -16000.15634920429
  -7320.469534664509
   7488.3306301311595
  24936.8576776472
  46772.052089843004
  81884.83427025029
  81884.83427025029

In [21]:
# Run tests to assess model fit
isapprox(est_model.B[1].β, model.B[1].β, atol=0.1) || isapprox(est_model.B[1].β, model.B[2].β, atol=0.1)
isapprox(est_model.B[2].β, model.B[2].β, atol=0.1) || isapprox(est_model.B[2].β, model.B[1].β, atol=0.1)

true